In [1]:
%%time

import datetime as dt
import os

import pandas
import pandas as pd
import praw
from praw.models import ListingGenerator
from tqdm import tqdm

from common.captioning.caption import BlipCaption
from common.schemas.pyarrow_schema import schema
from common.storage.azure_file_storage import AzureFileStorageAdapter

tqdm.pandas(desc="Progress")

file_system = AzureFileStorageAdapter('data').get_file_storage()

from common.data_frame_functions.functions import Functions

functions: Functions = Functions()

CPU times: total: 2.98 s
Wall time: 3 s


In [2]:
%%time

# "memes",	"CityPorn", "EarthPorn", "spaceporn",
subs = [
	"SFWRedheads",
	"sfwpetite",
	"SFWNextDoorGirls",
	"trippinthroughtime",
	"fatsquirrelhate",
	"itookapicture",
	"realasians",
	"KoreanHotties",
	"prettyasiangirls",
	"AsianOfficeLady",
	"mildlypenis",
	"AsianInvasion",
	"sexygirls",
	"PrettyGirls",
	"gentlemanboners",
	"hotofficegirls",
	"tightdresses",
	"DLAH",
	"bathandbodyworks"
]

CPU times: total: 0 ns
Wall time: 0 ns


In [3]:
%%time

sources = [
	{"name": "CityDiffusion", "data": ["CityPorn"]},
	{"name": "NatureDiffusion", "data": ["EarthPorn"]},
	{"name": "CosmicDiffusion", "data": ["spaceporn"]},
	{"name": "ITAPDiffusion", "data": ["itookapicture"]},
	{"name": "MemeDiffusion", "data": ["memes"]},
	{"name": "TTTDiffusion", "data": ["trippinthroughtime"]},
	{"name": "WallStreetDiffusion", "data": ["wallstreetbets"]},
	{"name": "SexyDiffusion", "data": ["selfies", "Amicute", "amihot", "AmIhotAF", "HotGirlNextDoor"]},
	{"name": "FatSquirrelDiffusion", "data": ["fatsquirrelhate"]},
	{"name": "CelebrityDiffusion", "data": ["celebrities"]},
	{"name": "OldLadyDiffusion", "data": ["oldladiesbakingpies"]},
	{"name": "SWFPetite", "data": ["sfwpetite"]},
	{"name": "SFWMilfs", "data": ["cougars_and_milfs_sfw"]},
	{"name": "RedHeadDiffusion", "data": ["SFWRedheads"]},
	{"name": "NextDoorGirlsDiffusion", "data": ["SFWNextDoorGirls"]},
	{"name": "SexyAsianDiffusion","data": ["realasians", "KoreanHotties", "prettyasiangirls", "AsianOfficeLady", "AsianInvasion"]},
	{"name": "MildlyPenisDiffusion", "data": ["mildlypenis"]},
	{"name": "PrettyGirlDiffusion","data": ["sexygirls", "PrettyGirls", "gentlemanboners", "hotofficegirls", "tightdresses", "DLAH"]},
	{"name": "CandleDiffusion", "data": ["bathandbodyworks"]}
]
sources_df = pd.DataFrame.from_records(sources)

CPU times: total: 0 ns
Wall time: 0 ns


In [4]:
%%time

extant_data = pandas.read_parquet("data/processed_raw_data.parquet", engine='pyarrow', filesystem=file_system,
								  schema=schema)
extant_data.set_index('id', inplace=True)
display(extant_data)

,subreddit,author,title,caption,hash,permalink,original_url,image_name,path,model,exists,curated,accept,tags
id,,,,,,,,,,,,,,
100rn7k,AmIhotAF,veritynicole,"hey, hows your new year going (23F)",a woman in a white shirt and black pants is po...,4bd00c19fa0ff2ade855e6d364b0760b,/r/AmIhotAF/comments/100rn7k/hey_hows_your_new...,https://i.redd.it/n7r47s0gkh9a1.jpg,100rn7k.jpg,data/image/100rn7k.jpg,SexyDiffusion,True,True,False,[]
1013bdt,AmIhotAF,RaulDea9286,36F - ITALIAN,arafed image of a woman in a bikini top,7c0d158cba8654ef1c635cbc5471d597,/r/AmIhotAF/comments/1013bdt/36f_italian/,https://i.redd.it/bg0wwdlt5k9a1.jpg,1013bdt.jpg,data/image/1013bdt.jpg,SexyDiffusion,True,True,True,[]
105mekt,AmIhotAF,lindaniz,interesting in good forward relationship (f24),a close up of a woman with red hair and a whit...,ba4a0962cca2266a741e1e1700589c04,/r/AmIhotAF/comments/105mekt/interesting_in_go...,https://i.redd.it/4avjshsz8naa1.jpg,105mekt.jpg,data/image/105mekt.jpg,SexyDiffusion,True,True,True,[]
105qvgl,AmIhotAF,CaitVLove11,Laughing is my favorite 😆,a woman in a blue tank top and shorts is smili...,27bfe82c37314a0bcf02ab72eaf3a9e5,/r/AmIhotAF/comments/105qvgl/laughing_is_my_fa...,https://i.redd.it/2pulzr0lxmaa1.jpg,105qvgl.jpg,data/image/105qvgl.jpg,SexyDiffusion,True,True,True,[]
105rpcj,AmIhotAF,Flashy-Desk1858,[f22] What do you think when you see me?,a woman in a blue bikini top and a blue bra top,329eb42b8267fa1cc2980da8e48bcef1,/r/AmIhotAF/comments/105rpcj/f22_what_do_you_t...,https://i.redd.it/rz68pf934naa1.jpg,105rpcj.jpg,data/image/105rpcj.jpg,SexyDiffusion,True,True,True,[]
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
131dso7,DLAH,Wallydinger123,Happy B Day,there are three women posing for a picture in ...,ebb4b4a88dc79e2f2f57b10f32bf03c0,/r/DLAH/comments/131dso7/happy_b_day/,https://i.redd.it/36gurz533jwa1.jpg,131dso7.jpg,data/image/131dso7.jpg,PrettyGirlDiffusion,True,True,False,[]
131eznp,DLAH,Long_dong111,Perfect,araffe woman in a white dress sitting on a pur...,5c944c6d62b1ae182c5a6a40c64195f4,/r/DLAH/comments/131eznp/perfect/,https://i.redd.it/dnbbx4zucjwa1.jpg,131eznp.jpg,data/image/131eznp.jpg,PrettyGirlDiffusion,True,True,True,[]
131g3am,DLAH,ThrownAwayMiles,DLAH left or right,two women in short dresses posing for a pictur...,ab1f8fc33c85f4cfce12c810cf22229e,/r/DLAH/comments/131g3am/dlah_left_or_right/,https://i.redd.it/zpp70oor3lwa1.jpg,131g3am.jpg,data/image/131g3am.jpg,PrettyGirlDiffusion,True,True,True,[]


CPU times: total: 297 ms
Wall time: 978 ms


In [5]:
%%time

curated_data = pandas.read_parquet("data/parquet/back.parquet", engine='pyarrow', filesystem=file_system, schema=schema)
curated_data.set_index('id', inplace=True)

display("==== CURATED DATA ====")
display(curated_data.shape)
display(curated_data)

'==== CURATED DATA ===='

(36107, 14)

,subreddit,author,title,caption,hash,permalink,original_url,image_name,path,model,exists,curated,accept,tags
id,,,,,,,,,,,,,,
100rn7k,AmIhotAF,veritynicole,"hey, hows your new year going (23F)",a woman in a white shirt and black pants is po...,4bd00c19fa0ff2ade855e6d364b0760b,/r/AmIhotAF/comments/100rn7k/hey_hows_your_new...,https://i.redd.it/n7r47s0gkh9a1.jpg,100rn7k.jpg,data/image/100rn7k.jpg,SexyDiffusion,True,True,False,[]
1013bdt,AmIhotAF,RaulDea9286,36F - ITALIAN,arafed image of a woman in a bikini top,7c0d158cba8654ef1c635cbc5471d597,/r/AmIhotAF/comments/1013bdt/36f_italian/,https://i.redd.it/bg0wwdlt5k9a1.jpg,1013bdt.jpg,data/image/1013bdt.jpg,SexyDiffusion,True,True,True,[]
105mekt,AmIhotAF,lindaniz,interesting in good forward relationship (f24),a close up of a woman with red hair and a whit...,ba4a0962cca2266a741e1e1700589c04,/r/AmIhotAF/comments/105mekt/interesting_in_go...,https://i.redd.it/4avjshsz8naa1.jpg,105mekt.jpg,data/image/105mekt.jpg,SexyDiffusion,True,True,True,[]
105qvgl,AmIhotAF,CaitVLove11,Laughing is my favorite 😆,a woman in a blue tank top and shorts is smili...,27bfe82c37314a0bcf02ab72eaf3a9e5,/r/AmIhotAF/comments/105qvgl/laughing_is_my_fa...,https://i.redd.it/2pulzr0lxmaa1.jpg,105qvgl.jpg,data/image/105qvgl.jpg,SexyDiffusion,True,True,True,[]
105rpcj,AmIhotAF,Flashy-Desk1858,[f22] What do you think when you see me?,a woman in a blue bikini top and a blue bra top,329eb42b8267fa1cc2980da8e48bcef1,/r/AmIhotAF/comments/105rpcj/f22_what_do_you_t...,https://i.redd.it/rz68pf934naa1.jpg,105rpcj.jpg,data/image/105rpcj.jpg,SexyDiffusion,True,True,True,[]
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
13a86yn,bathandbodyworks,That_Animal6354,Gourmands are my favorite if you couldn't tell.,three different types of body care products ar...,07059e37cc477cf7d9121bcc631f197e,/r/bathandbodyworks/comments/13a86yn/gourmands...,https://i.redd.it/bvae4tc0pcya1.jpg,13a86yn.jpg,data/image/13a86yn.jpg,CandleDiffusion,True,False,False,[]
13a9xed,bathandbodyworks,hill-top,April Empties! 🥳,bottles of various types of skin care products...,7d7dc8a74cafaba5524844909b9319cd,/r/bathandbodyworks/comments/13a9xed/april_emp...,https://i.redd.it/n3h2v4qh2dya1.jpg,13a9xed.jpg,data/image/13a9xed.jpg,CandleDiffusion,True,False,False,[]
13abmo0,bathandbodyworks,dekudoll,"February, March, & April Empties!",a close up of a variety of body care products ...,d2424a18844842421e1baeca4c491a28,/r/bathandbodyworks/comments/13abmo0/february_...,https://i.redd.it/29cq0ftmfdya1.jpg,13abmo0.jpg,data/image/13abmo0.jpg,CandleDiffusion,True,False,False,[]


CPU times: total: 312 ms
Wall time: 664 ms


In [6]:
%%time

extant_data.update(curated_data)
updated_extant_data = extant_data.reset_index()

display("==== UPDATED EXTANT DATA ====")
display(updated_extant_data.shape)
display(updated_extant_data)
updated_extant_data.to_parquet("data/processed_raw_data.parquet", engine='pyarrow', filesystem=file_system, schema=schema)

'==== UPDATED EXTANT DATA ===='

(41505, 15)

,id,subreddit,author,title,caption,hash,permalink,original_url,image_name,path,model,exists,curated,accept,tags
0,100rn7k,AmIhotAF,veritynicole,"hey, hows your new year going (23F)",a woman in a white shirt and black pants is po...,4bd00c19fa0ff2ade855e6d364b0760b,/r/AmIhotAF/comments/100rn7k/hey_hows_your_new...,https://i.redd.it/n7r47s0gkh9a1.jpg,100rn7k.jpg,data/image/100rn7k.jpg,SexyDiffusion,True,True,False,[]
1,1013bdt,AmIhotAF,RaulDea9286,36F - ITALIAN,arafed image of a woman in a bikini top,7c0d158cba8654ef1c635cbc5471d597,/r/AmIhotAF/comments/1013bdt/36f_italian/,https://i.redd.it/bg0wwdlt5k9a1.jpg,1013bdt.jpg,data/image/1013bdt.jpg,SexyDiffusion,True,True,True,[]
2,105mekt,AmIhotAF,lindaniz,interesting in good forward relationship (f24),a close up of a woman with red hair and a whit...,ba4a0962cca2266a741e1e1700589c04,/r/AmIhotAF/comments/105mekt/interesting_in_go...,https://i.redd.it/4avjshsz8naa1.jpg,105mekt.jpg,data/image/105mekt.jpg,SexyDiffusion,True,True,True,[]
3,105qvgl,AmIhotAF,CaitVLove11,Laughing is my favorite 😆,a woman in a blue tank top and shorts is smili...,27bfe82c37314a0bcf02ab72eaf3a9e5,/r/AmIhotAF/comments/105qvgl/laughing_is_my_fa...,https://i.redd.it/2pulzr0lxmaa1.jpg,105qvgl.jpg,data/image/105qvgl.jpg,SexyDiffusion,True,True,True,[]
4,105rpcj,AmIhotAF,Flashy-Desk1858,[f22] What do you think when you see me?,a woman in a blue bikini top and a blue bra top,329eb42b8267fa1cc2980da8e48bcef1,/r/AmIhotAF/comments/105rpcj/f22_what_do_you_t...,https://i.redd.it/rz68pf934naa1.jpg,105rpcj.jpg,data/image/105rpcj.jpg,SexyDiffusion,True,True,True,[]
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
41500,131dso7,DLAH,Wallydinger123,Happy B Day,there are three women posing for a picture in ...,ebb4b4a88dc79e2f2f57b10f32bf03c0,/r/DLAH/comments/131dso7/happy_b_day/,https://i.redd.it/36gurz533jwa1.jpg,131dso7.jpg,data/image/131dso7.jpg,PrettyGirlDiffusion,True,True,False,[]
41501,131eznp,DLAH,Long_dong111,Perfect,araffe woman in a white dress sitting on a pur...,5c944c6d62b1ae182c5a6a40c64195f4,/r/DLAH/comments/131eznp/perfect/,https://i.redd.it/dnbbx4zucjwa1.jpg,131eznp.jpg,data/image/131eznp.jpg,PrettyGirlDiffusion,True,True,True,[]
41502,131g3am,DLAH,ThrownAwayMiles,DLAH left or right,two women in short dresses posing for a pictur...,ab1f8fc33c85f4cfce12c810cf22229e,/r/DLAH/comments/131g3am/dlah_left_or_right/,https://i.redd.it/zpp70oor3lwa1.jpg,131g3am.jpg,data/image/131g3am.jpg,PrettyGirlDiffusion,True,True,True,[]
41503,131jvcx,DLAH,robok212,Good morning 😍,araffe in a pink dress posing on a white wall,01f5058855526ef25888cff88065db4c,/r/DLAH/comments/131jvcx/good_morning/,https://i.redd.it/ozmbfhjv3mwa1.jpg,131jvcx.jpg,data/image/131jvcx.jpg,PrettyGirlDiffusion,True,True,False,[]


CPU times: total: 297 ms
Wall time: 3.17 s


In [7]:
%%time

reddit: praw.Reddit = praw.Reddit(client_id='5hVavL0PIRyM_1JSvqT6UQ', client_secret='BjD2kS3WNLnJc59RKY-JJUuc_Z9-JA', user_agent='script:%(bot_name)s:v%(bot_version)s (by /u/%(bot_author)s)')

CPU times: total: 0 ns
Wall time: 0 ns


In [8]:
for sub in tqdm(subs, total=len(subs), desc="Creating Temp Dir For Subs..."):
	temp_dir_path_ = f"temp/{sub}"
	if not os.path.exists(temp_dir_path_):
		os.makedirs(temp_dir_path_)
	os.makedirs(temp_dir_path_, exist_ok=True)

Creating Temp Dir For Subs...: 100%|██████████| 19/19 [00:00<?, ?it/s]


In [9]:
with open('log.txt', 'a') as f:
	for time_filter in ['week', 'day']:
		for sub in tqdm(subs, desc=f"{time_filter}", total=len(subs)):
			temp_dir_path = f"temp/{sub}"
			try:
				subreddit_stream: ListingGenerator = reddit.subreddit(display_name=sub).top(limit=100, time_filter=time_filter)
				subreddit_stream = list(subreddit_stream)
				for submission in tqdm(subreddit_stream, total=len(subreddit_stream),
									   desc=f"Posts - {sub} - {time_filter}"):
					if submission is None:
						continue
					else:
						if submission.id in extant_data.index.values:
							continue
					try:
						author_name = 'Unknown'
						subreddit_name = sub
						try:
							author_name = submission.author.name
						except Exception as e:
							author_name = 'Unknown'
							pass
						p = {
							'id': submission.id,
							'subreddit': subreddit_name,
							'author': author_name,
							'title': submission.title,
							'caption': '',
							'hash': '',
							'permalink': submission.permalink,
							'original_url': submission.url,
							'image_name': '',
							'path': '',
							'thumbnail_path': '',
							'exists': False,
							'curated': False,
							'Tags': ['']
						}
						pd.DataFrame([p]).to_parquet(f"{temp_dir_path}/{submission.id}.parquet")
					except Exception as e:
						log = functions.write_log_message(submission.id, sub, "Error Writing Post", e)
						f.write(log)
						continue
			except Exception as e:
				log = functions.write_log_message(submission.id, sub, f"Error Getting Posts For SubReddit", e)
				f.write(log)
				continue

Posts - AsianInvasion - day: 100%|██████████| 2/2 [00:00<?, ?it/s]

day: 100%|██████████| 19/19 [00:12<00:00,  1.58it/s]


In [10]:
%%time

data = []
for sub in tqdm(subs, desc="Reading Temp Dir For Subs..."):
	df = pandas.read_parquet(f"temp/{sub}", schema=schema, engine='pyarrow')
	records = df.to_dict(orient='records')
	data.extend(records)

temp_data = pandas.DataFrame(data)

for i, r in temp_data.iterrows():
	temp_data.loc[i, 'image_name'] = r.id + ".jpg"
	temp_data.loc[i, 'path'] = ""
	temp_data.loc[i, 'hash'] = ""
	temp_data.loc[i, 'caption'] = ""
	temp_data.loc[i, 'model'] = ""
	temp_data.loc[i, 'exists'] = False
	temp_data.loc[i, 'curated'] = False
	temp_data.loc[i, 'accept'] = False
	temp_data.loc[i, 'tags'] = ['']

temp_data.set_index('id', inplace=True, drop=False)

display("==== TEMP DATA ====")
display(temp_data.shape)
display(temp_data)

Reading Temp Dir For Subs...: 100%|██████████| 19/19 [00:00<00:00, 26.95it/s]


'==== TEMP DATA ===='

(1069, 15)

,id,subreddit,author,title,caption,hash,permalink,original_url,image_name,path,model,exists,curated,accept,tags
id,,,,,,,,,,,,,,,
133garc,133garc,SFWRedheads,festivalfashionista,In my fave uniform,,,/r/SFWRedheads/comments/133garc/in_my_fave_uni...,https://i.imgur.com/O2CHhag.gifv,133garc.jpg,,,False,False,False,[]
13571tn,13571tn,SFWRedheads,Anikaprincess,Sexy or no?,,,/r/SFWRedheads/comments/13571tn/sexy_or_no/,https://i.imgur.com/1wUudk8.gifv,13571tn.jpg,,,False,False,False,[]
135ezva,135ezva,SFWRedheads,Historical_Ratio_294,Not usually a red lipstick kinda girl...,,,/r/SFWRedheads/comments/135ezva/not_usually_a_...,https://i.redd.it/4nb3ruke5fxa1.png,135ezva.jpg,,,False,False,False,[]
135g58p,135g58p,SFWRedheads,Scarlett_somebody,Sunshine ☀️,,,/r/SFWRedheads/comments/135g58p/sunshine/,https://www.reddit.com/gallery/135g58p,135g58p.jpg,,,False,False,False,[]
1360008,1360008,SFWRedheads,MalloyPanda,Sweater for Chilly Spring Weather,,,/r/SFWRedheads/comments/1360008/sweater_for_ch...,https://v.redd.it/md4388wcwixa1,1360008.jpg,,,False,False,False,[]
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
13ai5nu,13ai5nu,bathandbodyworks,Jopaypay,Fairytale vs Dream Bright,,,/r/bathandbodyworks/comments/13ai5nu/fairytale...,https://www.reddit.com/r/bathandbodyworks/comm...,13ai5nu.jpg,,,False,False,False,[]
13aiopr,13aiopr,bathandbodyworks,SimpleOk1719,How long for a plug in to feel warm?,,,/r/bathandbodyworks/comments/13aiopr/how_long_...,https://www.reddit.com/r/bathandbodyworks/comm...,13aiopr.jpg,,,False,False,False,[]
13ajo0r,13ajo0r,bathandbodyworks,singshopsleep,All of Today's Offers,,,/r/bathandbodyworks/comments/13ajo0r/all_of_to...,https://i.redd.it/8tlent17ffya1.jpg,13ajo0r.jpg,,,False,False,False,[]


CPU times: total: 4.92 s
Wall time: 2.03 s


In [11]:
%%time

filtered = temp_data[~temp_data['id'].isin(extant_data.index.values) & temp_data['original_url'].str.endswith('.jpg') & (~temp_data['id'].isin(curated_data.index.values))]
filtered.dropna(axis=1, how='all')
filtered.reset_index(drop=True, inplace=True)
filtered.set_index('id', inplace=True, drop=False)
display("==== FILTERED DATA ====")
display(filtered.shape)
display(filtered)

'==== FILTERED DATA ===='

(83, 15)

,id,subreddit,author,title,caption,hash,permalink,original_url,image_name,path,model,exists,curated,accept,tags
id,,,,,,,,,,,,,,,
13828y6,13828y6,SFWRedheads,Just_RedFox,Am I your type? (oc),,,/r/SFWRedheads/comments/13828y6/am_i_your_type...,https://i.redd.it/a5e157nzmxxa1.jpg,13828y6.jpg,,,False,False,False,[]
13anyfl,13anyfl,SFWRedheads,mghnbarrett,Red on a redhead never gets old. Also it’s my ...,,,/r/SFWRedheads/comments/13anyfl/red_on_a_redhe...,https://i.redd.it/pyfxt9nregya1.jpg,13anyfl.jpg,,,False,False,False,[]
13aof2y,13aof2y,SFWRedheads,Brooklyn102219,Do you want to play?,,,/r/SFWRedheads/comments/13aof2y/do_you_want_to...,https://i.redd.it/0gpo09d6igya1.jpg,13aof2y.jpg,,,False,False,False,[]
13aox2z,13aox2z,SFWRedheads,Fantastic-Mix-5761,I feel great after a night of crazy dreams haha,,,/r/SFWRedheads/comments/13aox2z/i_feel_great_a...,https://i.redd.it/d6y5tf02lgya1.jpg,13aox2z.jpg,,,False,False,False,[]
13ap2w8,13ap2w8,SFWRedheads,a-subreddit,"tried on my prom dress again, lmk what u think...",,,/r/SFWRedheads/comments/13ap2w8/tried_on_my_pr...,https://i.imgur.com/fwQiGfa.jpg,13ap2w8.jpg,,,False,False,False,[]
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
13arvwg,13arvwg,tightdresses,arrayoutofbounds,Holiday nights,,,/r/tightdresses/comments/13arvwg/holiday_nights/,https://i.redd.it/05gjqjl4tgya1.jpg,13arvwg.jpg,,,False,False,False,[]
13anz14,13anz14,DLAH,ThisActuator3213,Black Dress,,,/r/DLAH/comments/13anz14/black_dress/,https://i.redd.it/em9b27jcxeya1.jpg,13anz14.jpg,,,False,False,False,[]
13560uf,13560uf,bathandbodyworks,seaweedsquirrel,April empties,,,/r/bathandbodyworks/comments/13560uf/april_emp...,https://i.redd.it/p9xffkdrrcxa1.jpg,13560uf.jpg,,,False,False,False,[]


CPU times: total: 31.2 ms
Wall time: 15.6 ms


In [12]:
%%time

filtered['model'] = filtered.progress_apply(lambda x: functions.add_source(x, sources), axis=1)

display("== Filtered Data With Model ==")
display(filtered.shape)
display(filtered)

Progress: 100%|██████████| 83/83 [00:00<?, ?it/s]
<timed exec>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


'== Filtered Data With Model =='

(83, 15)

,id,subreddit,author,title,caption,hash,permalink,original_url,image_name,path,model,exists,curated,accept,tags
id,,,,,,,,,,,,,,,
13828y6,13828y6,SFWRedheads,Just_RedFox,Am I your type? (oc),,,/r/SFWRedheads/comments/13828y6/am_i_your_type...,https://i.redd.it/a5e157nzmxxa1.jpg,13828y6.jpg,,RedHeadDiffusion,False,False,False,[]
13anyfl,13anyfl,SFWRedheads,mghnbarrett,Red on a redhead never gets old. Also it’s my ...,,,/r/SFWRedheads/comments/13anyfl/red_on_a_redhe...,https://i.redd.it/pyfxt9nregya1.jpg,13anyfl.jpg,,RedHeadDiffusion,False,False,False,[]
13aof2y,13aof2y,SFWRedheads,Brooklyn102219,Do you want to play?,,,/r/SFWRedheads/comments/13aof2y/do_you_want_to...,https://i.redd.it/0gpo09d6igya1.jpg,13aof2y.jpg,,RedHeadDiffusion,False,False,False,[]
13aox2z,13aox2z,SFWRedheads,Fantastic-Mix-5761,I feel great after a night of crazy dreams haha,,,/r/SFWRedheads/comments/13aox2z/i_feel_great_a...,https://i.redd.it/d6y5tf02lgya1.jpg,13aox2z.jpg,,RedHeadDiffusion,False,False,False,[]
13ap2w8,13ap2w8,SFWRedheads,a-subreddit,"tried on my prom dress again, lmk what u think...",,,/r/SFWRedheads/comments/13ap2w8/tried_on_my_pr...,https://i.imgur.com/fwQiGfa.jpg,13ap2w8.jpg,,RedHeadDiffusion,False,False,False,[]
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
13arvwg,13arvwg,tightdresses,arrayoutofbounds,Holiday nights,,,/r/tightdresses/comments/13arvwg/holiday_nights/,https://i.redd.it/05gjqjl4tgya1.jpg,13arvwg.jpg,,PrettyGirlDiffusion,False,False,False,[]
13anz14,13anz14,DLAH,ThisActuator3213,Black Dress,,,/r/DLAH/comments/13anz14/black_dress/,https://i.redd.it/em9b27jcxeya1.jpg,13anz14.jpg,,PrettyGirlDiffusion,False,False,False,[]
13560uf,13560uf,bathandbodyworks,seaweedsquirrel,April empties,,,/r/bathandbodyworks/comments/13560uf/april_emp...,https://i.redd.it/p9xffkdrrcxa1.jpg,13560uf.jpg,,CandleDiffusion,False,False,False,[]


CPU times: total: 15.6 ms
Wall time: 15.6 ms


In [13]:
%%time

file_list = file_system.ls("data/image")

filtered['path'] = filtered.progress_apply(lambda x: functions.fetch_image(x, file_list, file_system), axis=1)

display("== Filtered Data With Path ==")
display(filtered.shape)
display(filtered)

Progress: 100%|██████████| 83/83 [00:02<00:00, 30.88it/s] 
<timed exec>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


'== Filtered Data With Path =='

(83, 15)

,id,subreddit,author,title,caption,hash,permalink,original_url,image_name,path,model,exists,curated,accept,tags
id,,,,,,,,,,,,,,,
13828y6,13828y6,SFWRedheads,Just_RedFox,Am I your type? (oc),,,/r/SFWRedheads/comments/13828y6/am_i_your_type...,https://i.redd.it/a5e157nzmxxa1.jpg,13828y6.jpg,data/image/13828y6.jpg,RedHeadDiffusion,False,False,False,[]
13anyfl,13anyfl,SFWRedheads,mghnbarrett,Red on a redhead never gets old. Also it’s my ...,,,/r/SFWRedheads/comments/13anyfl/red_on_a_redhe...,https://i.redd.it/pyfxt9nregya1.jpg,13anyfl.jpg,data/image/13anyfl.jpg,RedHeadDiffusion,False,False,False,[]
13aof2y,13aof2y,SFWRedheads,Brooklyn102219,Do you want to play?,,,/r/SFWRedheads/comments/13aof2y/do_you_want_to...,https://i.redd.it/0gpo09d6igya1.jpg,13aof2y.jpg,data/image/13aof2y.jpg,RedHeadDiffusion,False,False,False,[]
13aox2z,13aox2z,SFWRedheads,Fantastic-Mix-5761,I feel great after a night of crazy dreams haha,,,/r/SFWRedheads/comments/13aox2z/i_feel_great_a...,https://i.redd.it/d6y5tf02lgya1.jpg,13aox2z.jpg,data/image/13aox2z.jpg,RedHeadDiffusion,False,False,False,[]
13ap2w8,13ap2w8,SFWRedheads,a-subreddit,"tried on my prom dress again, lmk what u think...",,,/r/SFWRedheads/comments/13ap2w8/tried_on_my_pr...,https://i.imgur.com/fwQiGfa.jpg,13ap2w8.jpg,data/image/13ap2w8.jpg,RedHeadDiffusion,False,False,False,[]
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
13arvwg,13arvwg,tightdresses,arrayoutofbounds,Holiday nights,,,/r/tightdresses/comments/13arvwg/holiday_nights/,https://i.redd.it/05gjqjl4tgya1.jpg,13arvwg.jpg,data/image/13arvwg.jpg,PrettyGirlDiffusion,False,False,False,[]
13anz14,13anz14,DLAH,ThisActuator3213,Black Dress,,,/r/DLAH/comments/13anz14/black_dress/,https://i.redd.it/em9b27jcxeya1.jpg,13anz14.jpg,data/image/13anz14.jpg,PrettyGirlDiffusion,False,False,False,[]
13560uf,13560uf,bathandbodyworks,seaweedsquirrel,April empties,,,/r/bathandbodyworks/comments/13560uf/april_emp...,https://i.redd.it/p9xffkdrrcxa1.jpg,13560uf.jpg,data/image/13560uf.jpg,CandleDiffusion,False,False,False,[]


CPU times: total: 19.6 s
Wall time: 35.4 s


In [14]:
%%time

filtered['exists'] = filtered.progress_apply(lambda x: functions.set_exists(x), axis=1)

display("== Filtered Data With Exists ==")
display(filtered.shape)
display(filtered)

Progress: 100%|██████████| 83/83 [00:00<?, ?it/s]
<timed exec>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


'== Filtered Data With Exists =='

(83, 15)

,id,subreddit,author,title,caption,hash,permalink,original_url,image_name,path,model,exists,curated,accept,tags
id,,,,,,,,,,,,,,,
13828y6,13828y6,SFWRedheads,Just_RedFox,Am I your type? (oc),,,/r/SFWRedheads/comments/13828y6/am_i_your_type...,https://i.redd.it/a5e157nzmxxa1.jpg,13828y6.jpg,data/image/13828y6.jpg,RedHeadDiffusion,True,False,False,[]
13anyfl,13anyfl,SFWRedheads,mghnbarrett,Red on a redhead never gets old. Also it’s my ...,,,/r/SFWRedheads/comments/13anyfl/red_on_a_redhe...,https://i.redd.it/pyfxt9nregya1.jpg,13anyfl.jpg,data/image/13anyfl.jpg,RedHeadDiffusion,True,False,False,[]
13aof2y,13aof2y,SFWRedheads,Brooklyn102219,Do you want to play?,,,/r/SFWRedheads/comments/13aof2y/do_you_want_to...,https://i.redd.it/0gpo09d6igya1.jpg,13aof2y.jpg,data/image/13aof2y.jpg,RedHeadDiffusion,True,False,False,[]
13aox2z,13aox2z,SFWRedheads,Fantastic-Mix-5761,I feel great after a night of crazy dreams haha,,,/r/SFWRedheads/comments/13aox2z/i_feel_great_a...,https://i.redd.it/d6y5tf02lgya1.jpg,13aox2z.jpg,data/image/13aox2z.jpg,RedHeadDiffusion,True,False,False,[]
13ap2w8,13ap2w8,SFWRedheads,a-subreddit,"tried on my prom dress again, lmk what u think...",,,/r/SFWRedheads/comments/13ap2w8/tried_on_my_pr...,https://i.imgur.com/fwQiGfa.jpg,13ap2w8.jpg,data/image/13ap2w8.jpg,RedHeadDiffusion,True,False,False,[]
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
13arvwg,13arvwg,tightdresses,arrayoutofbounds,Holiday nights,,,/r/tightdresses/comments/13arvwg/holiday_nights/,https://i.redd.it/05gjqjl4tgya1.jpg,13arvwg.jpg,data/image/13arvwg.jpg,PrettyGirlDiffusion,True,False,False,[]
13anz14,13anz14,DLAH,ThisActuator3213,Black Dress,,,/r/DLAH/comments/13anz14/black_dress/,https://i.redd.it/em9b27jcxeya1.jpg,13anz14.jpg,data/image/13anz14.jpg,PrettyGirlDiffusion,True,False,False,[]
13560uf,13560uf,bathandbodyworks,seaweedsquirrel,April empties,,,/r/bathandbodyworks/comments/13560uf/april_emp...,https://i.redd.it/p9xffkdrrcxa1.jpg,13560uf.jpg,data/image/13560uf.jpg,CandleDiffusion,True,False,False,[]


CPU times: total: 31.2 ms
Wall time: 31.3 ms


In [15]:
%%time

filtered['hash'] = filtered.progress_apply(lambda x: functions.set_hash(x), axis=1)

display("== Filtered Data With Hash ==")
display(filtered.shape)
display(filtered)

Progress: 100%|██████████| 83/83 [00:00<00:00, 590.11it/s]
<timed exec>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


'== Filtered Data With Hash =='

(83, 15)

,id,subreddit,author,title,caption,hash,permalink,original_url,image_name,path,model,exists,curated,accept,tags
id,,,,,,,,,,,,,,,
13828y6,13828y6,SFWRedheads,Just_RedFox,Am I your type? (oc),,c2de79816802d5a44ff19702e79722ec,/r/SFWRedheads/comments/13828y6/am_i_your_type...,https://i.redd.it/a5e157nzmxxa1.jpg,13828y6.jpg,data/image/13828y6.jpg,RedHeadDiffusion,True,False,False,[]
13anyfl,13anyfl,SFWRedheads,mghnbarrett,Red on a redhead never gets old. Also it’s my ...,,07669db9ed8e7692c658ff918a84928a,/r/SFWRedheads/comments/13anyfl/red_on_a_redhe...,https://i.redd.it/pyfxt9nregya1.jpg,13anyfl.jpg,data/image/13anyfl.jpg,RedHeadDiffusion,True,False,False,[]
13aof2y,13aof2y,SFWRedheads,Brooklyn102219,Do you want to play?,,0587f6b17b6406dbef0679a55abf7a6e,/r/SFWRedheads/comments/13aof2y/do_you_want_to...,https://i.redd.it/0gpo09d6igya1.jpg,13aof2y.jpg,data/image/13aof2y.jpg,RedHeadDiffusion,True,False,False,[]
13aox2z,13aox2z,SFWRedheads,Fantastic-Mix-5761,I feel great after a night of crazy dreams haha,,175765cd5f2d9648b6cb4395d08f3842,/r/SFWRedheads/comments/13aox2z/i_feel_great_a...,https://i.redd.it/d6y5tf02lgya1.jpg,13aox2z.jpg,data/image/13aox2z.jpg,RedHeadDiffusion,True,False,False,[]
13ap2w8,13ap2w8,SFWRedheads,a-subreddit,"tried on my prom dress again, lmk what u think...",,c7c803c7eeac3615eb1da0dc439f3b59,/r/SFWRedheads/comments/13ap2w8/tried_on_my_pr...,https://i.imgur.com/fwQiGfa.jpg,13ap2w8.jpg,data/image/13ap2w8.jpg,RedHeadDiffusion,True,False,False,[]
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
13arvwg,13arvwg,tightdresses,arrayoutofbounds,Holiday nights,,cf7f625bf2831808747d3794f25c1e1e,/r/tightdresses/comments/13arvwg/holiday_nights/,https://i.redd.it/05gjqjl4tgya1.jpg,13arvwg.jpg,data/image/13arvwg.jpg,PrettyGirlDiffusion,True,False,False,[]
13anz14,13anz14,DLAH,ThisActuator3213,Black Dress,,3a1eb8cc868144cd404f5b8eb2b3f107,/r/DLAH/comments/13anz14/black_dress/,https://i.redd.it/em9b27jcxeya1.jpg,13anz14.jpg,data/image/13anz14.jpg,PrettyGirlDiffusion,True,False,False,[]
13560uf,13560uf,bathandbodyworks,seaweedsquirrel,April empties,,0fd06cb96793d1d14c37552a6b92f956,/r/bathandbodyworks/comments/13560uf/april_emp...,https://i.redd.it/p9xffkdrrcxa1.jpg,13560uf.jpg,data/image/13560uf.jpg,CandleDiffusion,True,False,False,[]


CPU times: total: 156 ms
Wall time: 156 ms


In [16]:
%%time

caption_0 = BlipCaption("cuda:0")
caption_1 = BlipCaption("cuda:1")

CPU times: total: 48.8 s
Wall time: 44.5 s


In [17]:
%%time

filtered['caption'] = filtered.progress_apply(lambda x: functions.apply_caption(x, [caption_0, caption_1]), axis=1)

display("== Filtered Data With Caption ==")
display(filtered.shape)
display(filtered)

Progress:   0%|          | 0/83 [00:00<?, ?it/s]D:\code\repos\simple-collection\venv\lib\site-packages\transformers\generation\utils.py:1313: UserWarning: Using `max_length`'s default (20) to control the generation length. This behaviour is deprecated and will be removed from the config in v5 of Transformers -- we recommend using `max_new_tokens` to control the maximum length of the generation.
  warnings.warn(
Progress: 100%|██████████| 83/83 [01:23<00:00,  1.00s/it]
<timed exec>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


'== Filtered Data With Caption =='

(83, 15)

,id,subreddit,author,title,caption,hash,permalink,original_url,image_name,path,model,exists,curated,accept,tags
id,,,,,,,,,,,,,,,
13828y6,13828y6,SFWRedheads,Just_RedFox,Am I your type? (oc),there is a woman with red hair and a blue top,c2de79816802d5a44ff19702e79722ec,/r/SFWRedheads/comments/13828y6/am_i_your_type...,https://i.redd.it/a5e157nzmxxa1.jpg,13828y6.jpg,data/image/13828y6.jpg,RedHeadDiffusion,True,False,False,[]
13anyfl,13anyfl,SFWRedheads,mghnbarrett,Red on a redhead never gets old. Also it’s my ...,arafed woman in a red bikini taking a selfie i...,07669db9ed8e7692c658ff918a84928a,/r/SFWRedheads/comments/13anyfl/red_on_a_redhe...,https://i.redd.it/pyfxt9nregya1.jpg,13anyfl.jpg,data/image/13anyfl.jpg,RedHeadDiffusion,True,False,False,[]
13aof2y,13aof2y,SFWRedheads,Brooklyn102219,Do you want to play?,there is a woman taking a selfie in a bathroom...,0587f6b17b6406dbef0679a55abf7a6e,/r/SFWRedheads/comments/13aof2y/do_you_want_to...,https://i.redd.it/0gpo09d6igya1.jpg,13aof2y.jpg,data/image/13aof2y.jpg,RedHeadDiffusion,True,False,False,[]
13aox2z,13aox2z,SFWRedheads,Fantastic-Mix-5761,I feel great after a night of crazy dreams haha,there is a woman that is standing up with her ...,175765cd5f2d9648b6cb4395d08f3842,/r/SFWRedheads/comments/13aox2z/i_feel_great_a...,https://i.redd.it/d6y5tf02lgya1.jpg,13aox2z.jpg,data/image/13aox2z.jpg,RedHeadDiffusion,True,False,False,[]
13ap2w8,13ap2w8,SFWRedheads,a-subreddit,"tried on my prom dress again, lmk what u think...",arafed woman in a pink dress posing in a bathroom,c7c803c7eeac3615eb1da0dc439f3b59,/r/SFWRedheads/comments/13ap2w8/tried_on_my_pr...,https://i.imgur.com/fwQiGfa.jpg,13ap2w8.jpg,data/image/13ap2w8.jpg,RedHeadDiffusion,True,False,False,[]
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
13arvwg,13arvwg,tightdresses,arrayoutofbounds,Holiday nights,araffe woman in a white dress leaning against ...,cf7f625bf2831808747d3794f25c1e1e,/r/tightdresses/comments/13arvwg/holiday_nights/,https://i.redd.it/05gjqjl4tgya1.jpg,13arvwg.jpg,data/image/13arvwg.jpg,PrettyGirlDiffusion,True,False,False,[]
13anz14,13anz14,DLAH,ThisActuator3213,Black Dress,araffe woman in a black dress sitting on a bla...,3a1eb8cc868144cd404f5b8eb2b3f107,/r/DLAH/comments/13anz14/black_dress/,https://i.redd.it/em9b27jcxeya1.jpg,13anz14.jpg,data/image/13anz14.jpg,PrettyGirlDiffusion,True,False,False,[]
13560uf,13560uf,bathandbodyworks,seaweedsquirrel,April empties,a close up of a towel with various types of bo...,0fd06cb96793d1d14c37552a6b92f956,/r/bathandbodyworks/comments/13560uf/april_emp...,https://i.redd.it/p9xffkdrrcxa1.jpg,13560uf.jpg,data/image/13560uf.jpg,CandleDiffusion,True,False,False,[]


CPU times: total: 2min 37s
Wall time: 1min 23s


In [18]:
%%time

filtered_more = filtered.loc[(filtered['caption'] != "") & (filtered['caption'].notnull()) & (filtered['exists'] == True)]

display("== Filtered On Missing Caption ==")
display(filtered_more.shape)
display(filtered_more)

'== Filtered On Missing Caption =='

(75, 15)

,id,subreddit,author,title,caption,hash,permalink,original_url,image_name,path,model,exists,curated,accept,tags
id,,,,,,,,,,,,,,,
13828y6,13828y6,SFWRedheads,Just_RedFox,Am I your type? (oc),there is a woman with red hair and a blue top,c2de79816802d5a44ff19702e79722ec,/r/SFWRedheads/comments/13828y6/am_i_your_type...,https://i.redd.it/a5e157nzmxxa1.jpg,13828y6.jpg,data/image/13828y6.jpg,RedHeadDiffusion,True,False,False,[]
13anyfl,13anyfl,SFWRedheads,mghnbarrett,Red on a redhead never gets old. Also it’s my ...,arafed woman in a red bikini taking a selfie i...,07669db9ed8e7692c658ff918a84928a,/r/SFWRedheads/comments/13anyfl/red_on_a_redhe...,https://i.redd.it/pyfxt9nregya1.jpg,13anyfl.jpg,data/image/13anyfl.jpg,RedHeadDiffusion,True,False,False,[]
13aof2y,13aof2y,SFWRedheads,Brooklyn102219,Do you want to play?,there is a woman taking a selfie in a bathroom...,0587f6b17b6406dbef0679a55abf7a6e,/r/SFWRedheads/comments/13aof2y/do_you_want_to...,https://i.redd.it/0gpo09d6igya1.jpg,13aof2y.jpg,data/image/13aof2y.jpg,RedHeadDiffusion,True,False,False,[]
13aox2z,13aox2z,SFWRedheads,Fantastic-Mix-5761,I feel great after a night of crazy dreams haha,there is a woman that is standing up with her ...,175765cd5f2d9648b6cb4395d08f3842,/r/SFWRedheads/comments/13aox2z/i_feel_great_a...,https://i.redd.it/d6y5tf02lgya1.jpg,13aox2z.jpg,data/image/13aox2z.jpg,RedHeadDiffusion,True,False,False,[]
13ap2w8,13ap2w8,SFWRedheads,a-subreddit,"tried on my prom dress again, lmk what u think...",arafed woman in a pink dress posing in a bathroom,c7c803c7eeac3615eb1da0dc439f3b59,/r/SFWRedheads/comments/13ap2w8/tried_on_my_pr...,https://i.imgur.com/fwQiGfa.jpg,13ap2w8.jpg,data/image/13ap2w8.jpg,RedHeadDiffusion,True,False,False,[]
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
13arvwg,13arvwg,tightdresses,arrayoutofbounds,Holiday nights,araffe woman in a white dress leaning against ...,cf7f625bf2831808747d3794f25c1e1e,/r/tightdresses/comments/13arvwg/holiday_nights/,https://i.redd.it/05gjqjl4tgya1.jpg,13arvwg.jpg,data/image/13arvwg.jpg,PrettyGirlDiffusion,True,False,False,[]
13anz14,13anz14,DLAH,ThisActuator3213,Black Dress,araffe woman in a black dress sitting on a bla...,3a1eb8cc868144cd404f5b8eb2b3f107,/r/DLAH/comments/13anz14/black_dress/,https://i.redd.it/em9b27jcxeya1.jpg,13anz14.jpg,data/image/13anz14.jpg,PrettyGirlDiffusion,True,False,False,[]
13560uf,13560uf,bathandbodyworks,seaweedsquirrel,April empties,a close up of a towel with various types of bo...,0fd06cb96793d1d14c37552a6b92f956,/r/bathandbodyworks/comments/13560uf/april_emp...,https://i.redd.it/p9xffkdrrcxa1.jpg,13560uf.jpg,data/image/13560uf.jpg,CandleDiffusion,True,False,False,[]


CPU times: total: 15.6 ms
Wall time: 15.8 ms


In [19]:
%%time

dropped = filtered_more.dropna(axis=1, how='all')
dropped.reset_index(drop=True, inplace=True)

display("== Dropped And Reset Data ==")
display(dropped.shape)
display(dropped)

'== Dropped And Reset Data =='

(75, 15)

,id,subreddit,author,title,caption,hash,permalink,original_url,image_name,path,model,exists,curated,accept,tags
0,13828y6,SFWRedheads,Just_RedFox,Am I your type? (oc),there is a woman with red hair and a blue top,c2de79816802d5a44ff19702e79722ec,/r/SFWRedheads/comments/13828y6/am_i_your_type...,https://i.redd.it/a5e157nzmxxa1.jpg,13828y6.jpg,data/image/13828y6.jpg,RedHeadDiffusion,True,False,False,[]
1,13anyfl,SFWRedheads,mghnbarrett,Red on a redhead never gets old. Also it’s my ...,arafed woman in a red bikini taking a selfie i...,07669db9ed8e7692c658ff918a84928a,/r/SFWRedheads/comments/13anyfl/red_on_a_redhe...,https://i.redd.it/pyfxt9nregya1.jpg,13anyfl.jpg,data/image/13anyfl.jpg,RedHeadDiffusion,True,False,False,[]
2,13aof2y,SFWRedheads,Brooklyn102219,Do you want to play?,there is a woman taking a selfie in a bathroom...,0587f6b17b6406dbef0679a55abf7a6e,/r/SFWRedheads/comments/13aof2y/do_you_want_to...,https://i.redd.it/0gpo09d6igya1.jpg,13aof2y.jpg,data/image/13aof2y.jpg,RedHeadDiffusion,True,False,False,[]
3,13aox2z,SFWRedheads,Fantastic-Mix-5761,I feel great after a night of crazy dreams haha,there is a woman that is standing up with her ...,175765cd5f2d9648b6cb4395d08f3842,/r/SFWRedheads/comments/13aox2z/i_feel_great_a...,https://i.redd.it/d6y5tf02lgya1.jpg,13aox2z.jpg,data/image/13aox2z.jpg,RedHeadDiffusion,True,False,False,[]
4,13ap2w8,SFWRedheads,a-subreddit,"tried on my prom dress again, lmk what u think...",arafed woman in a pink dress posing in a bathroom,c7c803c7eeac3615eb1da0dc439f3b59,/r/SFWRedheads/comments/13ap2w8/tried_on_my_pr...,https://i.imgur.com/fwQiGfa.jpg,13ap2w8.jpg,data/image/13ap2w8.jpg,RedHeadDiffusion,True,False,False,[]
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
70,13arvwg,tightdresses,arrayoutofbounds,Holiday nights,araffe woman in a white dress leaning against ...,cf7f625bf2831808747d3794f25c1e1e,/r/tightdresses/comments/13arvwg/holiday_nights/,https://i.redd.it/05gjqjl4tgya1.jpg,13arvwg.jpg,data/image/13arvwg.jpg,PrettyGirlDiffusion,True,False,False,[]
71,13anz14,DLAH,ThisActuator3213,Black Dress,araffe woman in a black dress sitting on a bla...,3a1eb8cc868144cd404f5b8eb2b3f107,/r/DLAH/comments/13anz14/black_dress/,https://i.redd.it/em9b27jcxeya1.jpg,13anz14.jpg,data/image/13anz14.jpg,PrettyGirlDiffusion,True,False,False,[]
72,13560uf,bathandbodyworks,seaweedsquirrel,April empties,a close up of a towel with various types of bo...,0fd06cb96793d1d14c37552a6b92f956,/r/bathandbodyworks/comments/13560uf/april_emp...,https://i.redd.it/p9xffkdrrcxa1.jpg,13560uf.jpg,data/image/13560uf.jpg,CandleDiffusion,True,False,False,[]
73,13aocf4,bathandbodyworks,1C3DC0FF33,Is gingham love getting discontinued soon?,a close up of a bottle of perfume on a white b...,764309345154649be478ae835668fc88,/r/bathandbodyworks/comments/13aocf4/is_gingha...,https://i.redd.it/ueb3lqgohgya1.jpg,13aocf4.jpg,data/image/13aocf4.jpg,CandleDiffusion,True,False,False,[]


CPU times: total: 15.6 ms
Wall time: 15.7 ms


In [20]:
%%time

current = pandas.read_parquet("data/parquet/back.parquet", engine='pyarrow', filesystem=file_system)

display(current.shape)
display(current)

(36107, 15)

,id,subreddit,author,title,caption,hash,permalink,original_url,image_name,path,model,exists,curated,accept,tags
0,100rn7k,AmIhotAF,veritynicole,"hey, hows your new year going (23F)",a woman in a white shirt and black pants is po...,4bd00c19fa0ff2ade855e6d364b0760b,/r/AmIhotAF/comments/100rn7k/hey_hows_your_new...,https://i.redd.it/n7r47s0gkh9a1.jpg,100rn7k.jpg,data/image/100rn7k.jpg,SexyDiffusion,True,True,False,[]
1,1013bdt,AmIhotAF,RaulDea9286,36F - ITALIAN,arafed image of a woman in a bikini top,7c0d158cba8654ef1c635cbc5471d597,/r/AmIhotAF/comments/1013bdt/36f_italian/,https://i.redd.it/bg0wwdlt5k9a1.jpg,1013bdt.jpg,data/image/1013bdt.jpg,SexyDiffusion,True,True,True,[]
2,105mekt,AmIhotAF,lindaniz,interesting in good forward relationship (f24),a close up of a woman with red hair and a whit...,ba4a0962cca2266a741e1e1700589c04,/r/AmIhotAF/comments/105mekt/interesting_in_go...,https://i.redd.it/4avjshsz8naa1.jpg,105mekt.jpg,data/image/105mekt.jpg,SexyDiffusion,True,True,True,[]
3,105qvgl,AmIhotAF,CaitVLove11,Laughing is my favorite 😆,a woman in a blue tank top and shorts is smili...,27bfe82c37314a0bcf02ab72eaf3a9e5,/r/AmIhotAF/comments/105qvgl/laughing_is_my_fa...,https://i.redd.it/2pulzr0lxmaa1.jpg,105qvgl.jpg,data/image/105qvgl.jpg,SexyDiffusion,True,True,True,[]
4,105rpcj,AmIhotAF,Flashy-Desk1858,[f22] What do you think when you see me?,a woman in a blue bikini top and a blue bra top,329eb42b8267fa1cc2980da8e48bcef1,/r/AmIhotAF/comments/105rpcj/f22_what_do_you_t...,https://i.redd.it/rz68pf934naa1.jpg,105rpcj.jpg,data/image/105rpcj.jpg,SexyDiffusion,True,True,True,[]
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
36102,13a86yn,bathandbodyworks,That_Animal6354,Gourmands are my favorite if you couldn't tell.,three different types of body care products ar...,07059e37cc477cf7d9121bcc631f197e,/r/bathandbodyworks/comments/13a86yn/gourmands...,https://i.redd.it/bvae4tc0pcya1.jpg,13a86yn.jpg,data/image/13a86yn.jpg,CandleDiffusion,True,False,False,[]
36103,13a9xed,bathandbodyworks,hill-top,April Empties! 🥳,bottles of various types of skin care products...,7d7dc8a74cafaba5524844909b9319cd,/r/bathandbodyworks/comments/13a9xed/april_emp...,https://i.redd.it/n3h2v4qh2dya1.jpg,13a9xed.jpg,data/image/13a9xed.jpg,CandleDiffusion,True,False,False,[]
36104,13abmo0,bathandbodyworks,dekudoll,"February, March, & April Empties!",a close up of a variety of body care products ...,d2424a18844842421e1baeca4c491a28,/r/bathandbodyworks/comments/13abmo0/february_...,https://i.redd.it/29cq0ftmfdya1.jpg,13abmo0.jpg,data/image/13abmo0.jpg,CandleDiffusion,True,False,False,[]
36105,13acr41,bathandbodyworks,notalivejustliving,Scent of the night,candles are sitting on a table next to a cake,1a7cc03534fa2e3068398700e8180485,/r/bathandbodyworks/comments/13acr41/scent_of_...,https://i.redd.it/3p5gzlfsodya1.jpg,13acr41.jpg,data/image/13acr41.jpg,CandleDiffusion,True,False,False,[]


CPU times: total: 234 ms
Wall time: 1.02 s


In [21]:
%%time

temp_current = current
temp_current.set_index('id', inplace=True, drop=False)

temp_new = dropped
temp_new.set_index('id', inplace=True, drop=False)

temp_new.loc[~temp_new.index.isin(temp_current.index)]

CPU times: total: 0 ns
Wall time: 0 ns


,id,subreddit,author,title,caption,hash,permalink,original_url,image_name,path,model,exists,curated,accept,tags
id,,,,,,,,,,,,,,,
13828y6,13828y6,SFWRedheads,Just_RedFox,Am I your type? (oc),there is a woman with red hair and a blue top,c2de79816802d5a44ff19702e79722ec,/r/SFWRedheads/comments/13828y6/am_i_your_type...,https://i.redd.it/a5e157nzmxxa1.jpg,13828y6.jpg,data/image/13828y6.jpg,RedHeadDiffusion,True,False,False,[]
13anyfl,13anyfl,SFWRedheads,mghnbarrett,Red on a redhead never gets old. Also it’s my ...,arafed woman in a red bikini taking a selfie i...,07669db9ed8e7692c658ff918a84928a,/r/SFWRedheads/comments/13anyfl/red_on_a_redhe...,https://i.redd.it/pyfxt9nregya1.jpg,13anyfl.jpg,data/image/13anyfl.jpg,RedHeadDiffusion,True,False,False,[]
13aof2y,13aof2y,SFWRedheads,Brooklyn102219,Do you want to play?,there is a woman taking a selfie in a bathroom...,0587f6b17b6406dbef0679a55abf7a6e,/r/SFWRedheads/comments/13aof2y/do_you_want_to...,https://i.redd.it/0gpo09d6igya1.jpg,13aof2y.jpg,data/image/13aof2y.jpg,RedHeadDiffusion,True,False,False,[]
13aox2z,13aox2z,SFWRedheads,Fantastic-Mix-5761,I feel great after a night of crazy dreams haha,there is a woman that is standing up with her ...,175765cd5f2d9648b6cb4395d08f3842,/r/SFWRedheads/comments/13aox2z/i_feel_great_a...,https://i.redd.it/d6y5tf02lgya1.jpg,13aox2z.jpg,data/image/13aox2z.jpg,RedHeadDiffusion,True,False,False,[]
13ap2w8,13ap2w8,SFWRedheads,a-subreddit,"tried on my prom dress again, lmk what u think...",arafed woman in a pink dress posing in a bathroom,c7c803c7eeac3615eb1da0dc439f3b59,/r/SFWRedheads/comments/13ap2w8/tried_on_my_pr...,https://i.imgur.com/fwQiGfa.jpg,13ap2w8.jpg,data/image/13ap2w8.jpg,RedHeadDiffusion,True,False,False,[]
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
13arvwg,13arvwg,tightdresses,arrayoutofbounds,Holiday nights,araffe woman in a white dress leaning against ...,cf7f625bf2831808747d3794f25c1e1e,/r/tightdresses/comments/13arvwg/holiday_nights/,https://i.redd.it/05gjqjl4tgya1.jpg,13arvwg.jpg,data/image/13arvwg.jpg,PrettyGirlDiffusion,True,False,False,[]
13anz14,13anz14,DLAH,ThisActuator3213,Black Dress,araffe woman in a black dress sitting on a bla...,3a1eb8cc868144cd404f5b8eb2b3f107,/r/DLAH/comments/13anz14/black_dress/,https://i.redd.it/em9b27jcxeya1.jpg,13anz14.jpg,data/image/13anz14.jpg,PrettyGirlDiffusion,True,False,False,[]
13560uf,13560uf,bathandbodyworks,seaweedsquirrel,April empties,a close up of a towel with various types of bo...,0fd06cb96793d1d14c37552a6b92f956,/r/bathandbodyworks/comments/13560uf/april_emp...,https://i.redd.it/p9xffkdrrcxa1.jpg,13560uf.jpg,data/image/13560uf.jpg,CandleDiffusion,True,False,False,[]


In [22]:
%%time

concat = pd.concat([current, dropped])

display("== Concatenated Data ==")
display(concat.shape)
display(concat)

'== Concatenated Data =='

(36182, 15)

,id,subreddit,author,title,caption,hash,permalink,original_url,image_name,path,model,exists,curated,accept,tags
id,,,,,,,,,,,,,,,
100rn7k,100rn7k,AmIhotAF,veritynicole,"hey, hows your new year going (23F)",a woman in a white shirt and black pants is po...,4bd00c19fa0ff2ade855e6d364b0760b,/r/AmIhotAF/comments/100rn7k/hey_hows_your_new...,https://i.redd.it/n7r47s0gkh9a1.jpg,100rn7k.jpg,data/image/100rn7k.jpg,SexyDiffusion,True,True,False,[]
1013bdt,1013bdt,AmIhotAF,RaulDea9286,36F - ITALIAN,arafed image of a woman in a bikini top,7c0d158cba8654ef1c635cbc5471d597,/r/AmIhotAF/comments/1013bdt/36f_italian/,https://i.redd.it/bg0wwdlt5k9a1.jpg,1013bdt.jpg,data/image/1013bdt.jpg,SexyDiffusion,True,True,True,[]
105mekt,105mekt,AmIhotAF,lindaniz,interesting in good forward relationship (f24),a close up of a woman with red hair and a whit...,ba4a0962cca2266a741e1e1700589c04,/r/AmIhotAF/comments/105mekt/interesting_in_go...,https://i.redd.it/4avjshsz8naa1.jpg,105mekt.jpg,data/image/105mekt.jpg,SexyDiffusion,True,True,True,[]
105qvgl,105qvgl,AmIhotAF,CaitVLove11,Laughing is my favorite 😆,a woman in a blue tank top and shorts is smili...,27bfe82c37314a0bcf02ab72eaf3a9e5,/r/AmIhotAF/comments/105qvgl/laughing_is_my_fa...,https://i.redd.it/2pulzr0lxmaa1.jpg,105qvgl.jpg,data/image/105qvgl.jpg,SexyDiffusion,True,True,True,[]
105rpcj,105rpcj,AmIhotAF,Flashy-Desk1858,[f22] What do you think when you see me?,a woman in a blue bikini top and a blue bra top,329eb42b8267fa1cc2980da8e48bcef1,/r/AmIhotAF/comments/105rpcj/f22_what_do_you_t...,https://i.redd.it/rz68pf934naa1.jpg,105rpcj.jpg,data/image/105rpcj.jpg,SexyDiffusion,True,True,True,[]
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
13arvwg,13arvwg,tightdresses,arrayoutofbounds,Holiday nights,araffe woman in a white dress leaning against ...,cf7f625bf2831808747d3794f25c1e1e,/r/tightdresses/comments/13arvwg/holiday_nights/,https://i.redd.it/05gjqjl4tgya1.jpg,13arvwg.jpg,data/image/13arvwg.jpg,PrettyGirlDiffusion,True,False,False,[]
13anz14,13anz14,DLAH,ThisActuator3213,Black Dress,araffe woman in a black dress sitting on a bla...,3a1eb8cc868144cd404f5b8eb2b3f107,/r/DLAH/comments/13anz14/black_dress/,https://i.redd.it/em9b27jcxeya1.jpg,13anz14.jpg,data/image/13anz14.jpg,PrettyGirlDiffusion,True,False,False,[]
13560uf,13560uf,bathandbodyworks,seaweedsquirrel,April empties,a close up of a towel with various types of bo...,0fd06cb96793d1d14c37552a6b92f956,/r/bathandbodyworks/comments/13560uf/april_emp...,https://i.redd.it/p9xffkdrrcxa1.jpg,13560uf.jpg,data/image/13560uf.jpg,CandleDiffusion,True,False,False,[]


CPU times: total: 62.5 ms
Wall time: 48.8 ms


In [23]:
%%time

file_list_ = file_system.ls("data/image")
concat['path'] = concat.progress_apply(lambda x: functions.fix_path(x, file_list), axis=1)

display("== Concatenated Data With Path ==")
display(concat.shape)
display(concat)

Progress: 100%|██████████| 36182/36182 [00:18<00:00, 1994.59it/s]


'== Concatenated Data With Path =='

(36182, 15)

,id,subreddit,author,title,caption,hash,permalink,original_url,image_name,path,model,exists,curated,accept,tags
id,,,,,,,,,,,,,,,
100rn7k,100rn7k,AmIhotAF,veritynicole,"hey, hows your new year going (23F)",a woman in a white shirt and black pants is po...,4bd00c19fa0ff2ade855e6d364b0760b,/r/AmIhotAF/comments/100rn7k/hey_hows_your_new...,https://i.redd.it/n7r47s0gkh9a1.jpg,100rn7k.jpg,data/image/100rn7k.jpg,SexyDiffusion,True,True,False,[]
1013bdt,1013bdt,AmIhotAF,RaulDea9286,36F - ITALIAN,arafed image of a woman in a bikini top,7c0d158cba8654ef1c635cbc5471d597,/r/AmIhotAF/comments/1013bdt/36f_italian/,https://i.redd.it/bg0wwdlt5k9a1.jpg,1013bdt.jpg,data/image/1013bdt.jpg,SexyDiffusion,True,True,True,[]
105mekt,105mekt,AmIhotAF,lindaniz,interesting in good forward relationship (f24),a close up of a woman with red hair and a whit...,ba4a0962cca2266a741e1e1700589c04,/r/AmIhotAF/comments/105mekt/interesting_in_go...,https://i.redd.it/4avjshsz8naa1.jpg,105mekt.jpg,data/image/105mekt.jpg,SexyDiffusion,True,True,True,[]
105qvgl,105qvgl,AmIhotAF,CaitVLove11,Laughing is my favorite 😆,a woman in a blue tank top and shorts is smili...,27bfe82c37314a0bcf02ab72eaf3a9e5,/r/AmIhotAF/comments/105qvgl/laughing_is_my_fa...,https://i.redd.it/2pulzr0lxmaa1.jpg,105qvgl.jpg,data/image/105qvgl.jpg,SexyDiffusion,True,True,True,[]
105rpcj,105rpcj,AmIhotAF,Flashy-Desk1858,[f22] What do you think when you see me?,a woman in a blue bikini top and a blue bra top,329eb42b8267fa1cc2980da8e48bcef1,/r/AmIhotAF/comments/105rpcj/f22_what_do_you_t...,https://i.redd.it/rz68pf934naa1.jpg,105rpcj.jpg,data/image/105rpcj.jpg,SexyDiffusion,True,True,True,[]
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
13arvwg,13arvwg,tightdresses,arrayoutofbounds,Holiday nights,araffe woman in a white dress leaning against ...,cf7f625bf2831808747d3794f25c1e1e,/r/tightdresses/comments/13arvwg/holiday_nights/,https://i.redd.it/05gjqjl4tgya1.jpg,13arvwg.jpg,data/image/13arvwg.jpg,PrettyGirlDiffusion,True,False,False,[]
13anz14,13anz14,DLAH,ThisActuator3213,Black Dress,araffe woman in a black dress sitting on a bla...,3a1eb8cc868144cd404f5b8eb2b3f107,/r/DLAH/comments/13anz14/black_dress/,https://i.redd.it/em9b27jcxeya1.jpg,13anz14.jpg,data/image/13anz14.jpg,PrettyGirlDiffusion,True,False,False,[]
13560uf,13560uf,bathandbodyworks,seaweedsquirrel,April empties,a close up of a towel with various types of bo...,0fd06cb96793d1d14c37552a6b92f956,/r/bathandbodyworks/comments/13560uf/april_emp...,https://i.redd.it/p9xffkdrrcxa1.jpg,13560uf.jpg,data/image/13560uf.jpg,CandleDiffusion,True,False,False,[]


CPU times: total: 18.2 s
Wall time: 18.2 s


In [24]:
%%time

back_up_name = f"data/parquet/back_{dt.datetime.timestamp(dt.datetime.now())}.parquet"
display(f"== Writing Back-Up {back_up_name} ==")
current.to_parquet(back_up_name, schema=schema, filesystem=file_system)

'== Writing Back-Up data/parquet/back_1683474816.063419.parquet =='

CPU times: total: 281 ms
Wall time: 3 s


In [25]:
%%time

concat.to_parquet("data/parquet/back.parquet", schema=schema, filesystem=file_system)
new = pd.read_parquet("data/parquet/back.parquet", engine='pyarrow', schema=schema, filesystem=file_system)

display("== New Data ==")
display(new.shape)
display(new)

'== New Data =='

(36182, 15)

,id,subreddit,author,title,caption,hash,permalink,original_url,image_name,path,model,exists,curated,accept,tags
0,100rn7k,AmIhotAF,veritynicole,"hey, hows your new year going (23F)",a woman in a white shirt and black pants is po...,4bd00c19fa0ff2ade855e6d364b0760b,/r/AmIhotAF/comments/100rn7k/hey_hows_your_new...,https://i.redd.it/n7r47s0gkh9a1.jpg,100rn7k.jpg,data/image/100rn7k.jpg,SexyDiffusion,True,True,False,[]
1,1013bdt,AmIhotAF,RaulDea9286,36F - ITALIAN,arafed image of a woman in a bikini top,7c0d158cba8654ef1c635cbc5471d597,/r/AmIhotAF/comments/1013bdt/36f_italian/,https://i.redd.it/bg0wwdlt5k9a1.jpg,1013bdt.jpg,data/image/1013bdt.jpg,SexyDiffusion,True,True,True,[]
2,105mekt,AmIhotAF,lindaniz,interesting in good forward relationship (f24),a close up of a woman with red hair and a whit...,ba4a0962cca2266a741e1e1700589c04,/r/AmIhotAF/comments/105mekt/interesting_in_go...,https://i.redd.it/4avjshsz8naa1.jpg,105mekt.jpg,data/image/105mekt.jpg,SexyDiffusion,True,True,True,[]
3,105qvgl,AmIhotAF,CaitVLove11,Laughing is my favorite 😆,a woman in a blue tank top and shorts is smili...,27bfe82c37314a0bcf02ab72eaf3a9e5,/r/AmIhotAF/comments/105qvgl/laughing_is_my_fa...,https://i.redd.it/2pulzr0lxmaa1.jpg,105qvgl.jpg,data/image/105qvgl.jpg,SexyDiffusion,True,True,True,[]
4,105rpcj,AmIhotAF,Flashy-Desk1858,[f22] What do you think when you see me?,a woman in a blue bikini top and a blue bra top,329eb42b8267fa1cc2980da8e48bcef1,/r/AmIhotAF/comments/105rpcj/f22_what_do_you_t...,https://i.redd.it/rz68pf934naa1.jpg,105rpcj.jpg,data/image/105rpcj.jpg,SexyDiffusion,True,True,True,[]
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
36177,13arvwg,tightdresses,arrayoutofbounds,Holiday nights,araffe woman in a white dress leaning against ...,cf7f625bf2831808747d3794f25c1e1e,/r/tightdresses/comments/13arvwg/holiday_nights/,https://i.redd.it/05gjqjl4tgya1.jpg,13arvwg.jpg,data/image/13arvwg.jpg,PrettyGirlDiffusion,True,False,False,[]
36178,13anz14,DLAH,ThisActuator3213,Black Dress,araffe woman in a black dress sitting on a bla...,3a1eb8cc868144cd404f5b8eb2b3f107,/r/DLAH/comments/13anz14/black_dress/,https://i.redd.it/em9b27jcxeya1.jpg,13anz14.jpg,data/image/13anz14.jpg,PrettyGirlDiffusion,True,False,False,[]
36179,13560uf,bathandbodyworks,seaweedsquirrel,April empties,a close up of a towel with various types of bo...,0fd06cb96793d1d14c37552a6b92f956,/r/bathandbodyworks/comments/13560uf/april_emp...,https://i.redd.it/p9xffkdrrcxa1.jpg,13560uf.jpg,data/image/13560uf.jpg,CandleDiffusion,True,False,False,[]
36180,13aocf4,bathandbodyworks,1C3DC0FF33,Is gingham love getting discontinued soon?,a close up of a bottle of perfume on a white b...,764309345154649be478ae835668fc88,/r/bathandbodyworks/comments/13aocf4/is_gingha...,https://i.redd.it/ueb3lqgohgya1.jpg,13aocf4.jpg,data/image/13aocf4.jpg,CandleDiffusion,True,False,False,[]


CPU times: total: 422 ms
Wall time: 3.39 s


In [26]:
!jupyter notebook stop

^C
